In [1]:
from langchain.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.cache import SQLiteCache #cashing at memory and sqlite
from langchain.globals import set_llm_cache

In [2]:
set_llm_cache(SQLiteCache("./testCache.db"))

In [3]:
ollama = ChatOllama(
    model = "mistral:latest",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [4]:
chef = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "give me recipe of {cuisine}"),
])
translator = ChatPromptTemplate.from_messages([
    ("system", "you are a translator. you must translate english to {language}"),
    ("human", "{output}"),
])

recipe_chain = chef | ollama
translate_chain = translator | ollama


In [10]:

engRecipe_to_anotherLangRecipe = {"output":recipe_chain} | translate_chain

translate_chain.invoke({
    "language":"korean"
})

engRecipe_to_anotherLangRecipe.invoke({
    "cuisine":"milk tea",
})

KeyError: 'output'

In [ ]:

# first chain
first_prompt = ChatPromptTemplate.from_template(
    "How much is {a} + {b}. Answer only witht the result\nResult: "
)
first_chain = first_prompt | ollama

# second_chain
second_prompt = ChatPromptTemplate.from_template(
    "How much is {a} times {c}"
)
second_chain = second_prompt | ollama

# all together
complete_chain = ({
        "c": first_chain, # save result in variable 'c'
        "a": itemgetter("a")  # get variable a for next chain
        }
    | second_chain # run second_chain with a and c as input
    )

complete_chain.invoke({'a': 2, 'b':3})
